In [2]:
import pandas as pd
import scipy.stats as scs
import numpy as np
from scipy.stats import chi2_contingency

## Q1: Probability of error

Write a method to calculate the probability that you make at least one error in a set of m hypothesis tests, given the specified alpha threshold, ie. the probability of at least one false positive.

In [11]:
# atleastone_error_rate(m,alpha) takes as input m (the number of independent hypothesis 
# tests conducted), and alpha, the significance threshold and returns the probablilty 
# of making at least one error in the m tests
import scipy.stats as st

def atleastone_error_rate(m,alpha):
    error_rate = 1-(1-alpha)**m
    return(error_rate)

# For example,
atleastone_error_rate(3,0.10) # -> 0.2709999999999999

0.2709999999999999

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


## Q2: A/B Testing

Consider the AB test results in the csv file 'ab_data_demo.csv'. The data consists of 2000 users. The column 'group' indicates whether the user is assigned to the control group (A) or the treatment group (B). The column 'premium_signed_up' indicates whether the user signed up for the premium service or not with a 1 or 0, respectively. 

We want to test whether changes made to the signup process (treatment B) for a premium account significantly increased the sign up rate. 

(a) Write a method to calculate the signup rate for both the control (A) and treatment (B) groups and return as a tuple.  

In [62]:
# get_signup_rate(filename) takes as input a csv file to read the data from, 
# and returns a tuple with:
# the calculated signup rate (1=sign up) for each of the two groups A and B.

def get_signup_rate(filename):
    
    data = pd.read_csv(filename)
    groupa_rate = (data[(data['group']=='A') & (data['premium_signed_up']==1)].count()[0])/(data[data['group']=='A'].count()[0])
    groupb_rate = (data[(data['group']=='B') & (data['premium_signed_up']==1)].count()[0])/(data[data['group']=='B'].count()[0])
    return(groupa_rate, groupb_rate)

# For example,
get_signup_rate('ab_data_demo.csv') # -> (0.08891108891108891, 0.021021021021021023)


(0.08891108891108891, 0.021021021021021023)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


(b) Write a method to test the hypothesis that the treatment (B) significantly increased the signup rate compared to the control (A). First, compute the contigency table with rows corresponding to groups A and B, and columns indicating sign-up (1) or not (0). Then, use a chi-square test with the specified alpha, and return the pvalue and associated decision (accept/reject).  

In [94]:
# test_hypothesis(filename, alpha) takes as input a csv file to read the data from, 
# and returns a tuple with:
# (1) the first element as the p value returned from the chi-square test, and 
# (2) the second element as a string which returns one of two values: "accept" or "reject", 
# indicating whether the hypothesis, using the specified alpha, is accepted or rejected.

# For example,
# test_hypothesis('ab_data_demo.csv', 0.05) -> (5.356126765709662e-11, 'reject')

def test_hypothesis(filename, alpha):
    
    data = pd.read_csv(filename)
    data_group = data.groupby(['group','premium_signed_up']).size()
    A0 = data_group.loc[('A',0)]
    A1 = data_group.loc[('A',1)]
    B0 = data_group.loc[('B',0)]
    B1 = data_group.loc[('B',1)]
    obs = np.array([[A0, A1],[B0, B1]])
    pval = chi2_contingency(obs)[1]
    if pval > alpha:
        result = 'accept'
    else:
        result = 'reject'
    return(pval, result)

test_hypothesis('ab_data_demo.csv', 0.05)
 

(5.356126765709662e-11, 'reject')

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


## Q3: Bonferroni correction

Write a method to calculate the adjusted alpha threshold to use when applying a Bonferroni correction to a set of m hypothesis tests, given the specified alpha threshold.

In [95]:
# bonferroni_alpha(m,alpha) takes as input m (the number of independent hypothesis 
# tests conducted), and alpha, the significance threshold and returns the adjusted 
# alpha threshold to use when applying a Bonferroni correction


def bonferroni_alpha(m,alpha):
    p_val = alpha/m
    return(p_val)

# For example,
bonferroni_alpha(3,0.10) # -> 0.03333333333333333

0.03333333333333333

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


## Q4: False Discovery Rate (FDR)

(a) Write a method to calculate the False Discovery Rate (FDR) from a given vector of p-values and specified alpha threshold. 

In [103]:
# calculate_FDR(p_values, alpha) takes as input a vector of p vaiues and specified 
# alpha, and returns the False Discovery Rate (FDR) as a floating point number. 

def calculate_FDR(p_values, alpha):
    
    count = 0
    for n in p_values:
        if n< alpha:
            count += 1
    FDR = len(p_values)*alpha/count
    return(FDR)
    
# For example,
p_values = [0.001, 0.008, 0.039, 0.041, 0.042, 0.06, 0.074, 0.205, 0.212, 0.216, 0.222, 0.251, 0.269, 0.275, 0.34,
           0.341, 0.384, 0.569, 0.594, 0.696, 0.762, 0.94, 0.942, 0.975, 0.986]
calculate_FDR(p_values, 0.10) # -> 0.35714285714285715


0.35714285714285715

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


(b) Write a method to calculate the adjusted alpha threshold to use to achieve the specified False Discovery Rate (FDR) with a given vector of p-values. 

In [105]:
# calculate_alpha_for_FDR(p_values, fdr) takes as input a vector of p vaiues and specified 
# False Discovery Rate, and returns the adjusted alpha that will achieve the target FDR. 

def calculate_alpha_for_FDR(p_values, fdr):
    targetFDR = fdr
    p_values.sort(reverse=True)
    numP = len(p_values)
    l = []
    targetIdx = -1
    for i in range(numP):
        numSig = numP - (i)
        currAlpha = p_values[i]
        f = (numP * currAlpha) / numSig
        l.append(f)
        if f <= targetFDR and targetIdx < 0:
            targetIdx = i
    return(p_values[targetIdx])
    
    
# For example,
p_values = [0.001, 0.008, 0.039, 0.041, 0.042, 0.06, 0.074, 0.205, 0.212, 0.216, 0.222, 0.251, 0.269, 0.275, 0.34,
           0.341, 0.384, 0.569, 0.594, 0.696, 0.762, 0.94, 0.942, 0.975, 0.986]
calculate_alpha_for_FDR(p_values, 0.25) # -> 0.06


0.06

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


## Q5: Multiple hypothesis tests

Consider the file 'doc_term_demo.csv'. Each column corresponds to a word and each row corresponds to a portition of the Wikepedia document describing a university. There is a header row specifying the words, and the first column records the document name. The value in the cell (i,j) specifies the number of times word j is used in document i. 

(a) Write a method to read in the data and test the hypothesis that the counts for a specified pair of words differs significantly different across a specified pair of schools. 

For example, for the pair of schools 'purdue.txt','cornell.txt' and the pair of words 'research' and 'university', construct a contingency table with rows corresponding to the schools and columns corresponding to the words, with the cells recording the words counts in each school documnent. Then use a chi-square test to assess whether there's a signifcant difference in the observed counts. 

In [10]:
# doc_term_hypothesis(filename, word1, word2, school1, school2, alpha) takes as input 
# a csv file to read the data from, and returns a tuple with:
# (1) the first element as the p value returned from the chi-square test, 
# comparing the word coutns for the two schools, and 
# (2) the second element as a string which returns one of two values: "accept" or "reject", 
# indicating whether the hypothesis, using the specified alpha, is accepted or rejected.


filename = 'doc_term_demo.csv'

def doc_term_hypothesis(filename,word1,word2,school1,school2,alpha):
    
    data = pd.read_csv(filename, index_col=0)
    cor_uni = data.loc[school1,word1]
    cor_research = data.loc[school1,word2]
    pur_uni = data.loc[school2,word1]
    pur_research = data.loc[school2,word2]
    obs = np.array([[cor_uni, cor_research],[pur_uni, pur_research]])
    pval = chi2_contingency(obs)[1]
    if pval > alpha:
        result = 'accept'
    else:
        result = 'reject'
    return(pval, result)

# For example,
doc_term_hypothesis('doc_term_demo.csv','university','research','purdue.txt','cornell.txt',0.1)
# -> (0.4970314541039976, 'accept')


(0.4970314541039976, 'accept')

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


(b) Write a method to read in the data and test multiple hypothesese for a specified pair of schools. Consider all possible pairs of words, and test the hypothesis that the words counts differ significantly across the two schools, using the specified alpha threshold.

Note: only conduct tests on words that have non-zero values for both schools. I.e., every row and column in the contingency table should sum to >0.

Return a tuple with (i) the total number of tests conducted, and (ii) the number of significant tests.

In [65]:
# school_term_hypotheses(filename, school1, school2, alpha) takes as input 
# a csv file to read the data from, and returns a tuple with:
# (1) the total number of chi-square tests conducted when comparing all word pairs 
# for the two schools, and 
# (2) the number of tests that were significant, using the specified alpha.

filename = 'doc_term_demo.csv'

def school_term_hypotheses(filename,school1,school2,alpha):
    
    data = pd.read_csv(filename, index_col=0)
    school = data.loc[[school1,school2],:]
    school_drop_zero = school.replace(0,np.nan).dropna(axis=1,how="all")
    data_clean = school_drop_zero.replace(np.nan,0)
    significant = 0
    trial = 0
    for i in range(len(data_clean.columns)):
        for j in range(i + 1, len(data_clean.columns)):
            selected_df = data_clean.iloc[:,[i,j]]
            if selected_df.iloc[0, 0] + selected_df.iloc[0, 1] == 0 or selected_df.iloc[1, 0] + selected_df.iloc[1, 1] == 0:
                continue
            pval = chi2_contingency(selected_df)[1]
            trial += 1
            if pval < alpha:
                significant += 1
    return(trial, significant)

        
# For example,
school_term_hypotheses('doc_term_demo.csv','purdue.txt','uiuc.txt',0.05)
# -> (62, 41)


(62, 41)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


(c) Repeat Q5b, but use a Bonferroni correction when assessing significance.

Return a tuple with (i) the adjusted alpha threshold after applying the Bonferroni correction, and (ii) the number of significant tests.

In [66]:
# school_term_hypotheses_BC(filename, school1, school2, alpha) takes as input 
# a csv file to read the data from, and returns a tuple with:
# (1) the adjusted alpha threshold after applying the Bonferroni correction, and 
# (2) the number of tests that were significant, using the specified alpha and 
# a Bonferonni correction.

filename = 'doc_term_demo.csv'

def school_term_hypotheses_BC(filename,school1,school2,alpha):
    
    
    data = pd.read_csv(filename, index_col=0)
    school = data.loc[[school1,school2],:]
    school_drop_zero = school.replace(0,np.nan).dropna(axis=1,how="all")
    data_clean = school_drop_zero.replace(np.nan,0)
    significant = 0
    trial = 0
    pvalues = []
    for i in range(len(data_clean.columns)):
        for j in range(i + 1, len(data_clean.columns)):
            selected_df = data_clean.iloc[:,[i,j]]
            if selected_df.iloc[0, 0] + selected_df.iloc[0, 1] == 0 or selected_df.iloc[1, 0] + selected_df.iloc[1, 1] == 0:
                continue
            pval = chi2_contingency(selected_df)[1]
            trial += 1
            pvalues.append(pval)
            if pval < alpha:
                significant += 1
    Bonferroni = alpha/trial
    new_significant = 0
    for val in pvalues:
        if val < Bonferroni:
            new_significant += 1
        
    return(Bonferroni, new_significant)


# For example,
school_term_hypotheses_BC('doc_term_demo.csv','purdue.txt','uiuc.txt',0.05)
# -> (0.0008064516129032258, 23)

(0.0008064516129032258, 23)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


(d) Repeat Q5b to test multiple hypotheses, but take as input a specified FDR rate and return the corresponding adjusted alpha threshold.

Return a tuple with (i) the adjusted alpha threshold for the target FDR rate, and (ii) the number of significant tests.

In [64]:
# school_term_hypotheses_FDR(filename, school1, school2, targetFDR) takes as input 
# a csv file to read the data from, and returns a tuple with:
# (1) the adjusted alpha threshold to meet the target FDR rate, and 
# (2) the number of tests that were significant, using the FDR correction.

filename = 'doc_term_demo.csv'

def school_term_hypotheses_FDR(filename,school1,school2,targetFDR):
    
    data = pd.read_csv(filename, index_col=0)
    school = data.loc[[school1,school2],:]
    school_drop_zero = school.replace(0,np.nan).dropna(axis=1,how="all")
    data_clean = school_drop_zero.replace(np.nan,0)
    significant = 0
    trial = 0
    pvalues = []
    for i in range(len(data_clean.columns)):
        for j in range(i + 1, len(data_clean.columns)):
            selected_df = data_clean.iloc[:,[i,j]]
            if selected_df.iloc[0, 0] + selected_df.iloc[0, 1] == 0 or selected_df.iloc[1, 0] + selected_df.iloc[1, 1] == 0:
                continue
            pval = chi2_contingency(selected_df)[1]
            trial += 1
            pvalues.append(pval)
#             if pval < alpha:
#                 significant += 1
                
    targetFDR = targetFDR            
    pvalues.sort(reverse=True)
    numP = len(pvalues)
    l = []
    targetIdx = -1
    for i in range(numP):
        numSig = numP - (i)
        currAlpha = pvalues[i]
        f = (numP * currAlpha) / numSig
        l.append(f)
        if f <= targetFDR and targetIdx < 0:
            targetIdx = i
#     print(pvalues[targetIdx]) #adjusted alpha
    adj_alpha=pvalues[targetIdx]
    
    c=0
    for i in range(len(pvalues)):
        if pvalues[i]<adj_alpha:
            c=c+1
        
    return(adj_alpha, c)

# For example,
school_term_hypotheses_FDR('doc_term_demo.csv','purdue.txt','uiuc.txt',0.1)
# -> (0.011798507503906602, 39)

(0.011798507503906602, 39)

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
